# Classifying timephrases

This notebook will seek to establish a taxonomy of time phrases in Biblical Hebrew that is as comprehensive as possible. The `Construction` object is used as the starting point for the analysis. We already have a set of `Construction` objects (henceforth simply "cx") that have been preprocessed based on their subphrase grammar. These subphrases allow us to make certain selections of the data and place labels on the time phrases.

In [1]:
import collections
import pickle
import copy
import random
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from helpers import barplot_counts, convert2pandas
from tf_tools.load import load_tf
from tf_tools.tokenizers import tokenize_surface
from cx_analysis.cx import Construction
from cx_analysis.build import CXbuilder
from cx_analysis.search import SearchCX
from positions import Positions
from paths import cxs as cx_data
from paths import semvector

TF, api, A = load_tf()
F, E, T, L = api.F, api.E, api.T, api.L

with open(cx_data, 'rb') as infile:
    cx_load = pickle.load(infile)
    phrase2cxs = cx_load['phrase2cxs']
    
with open(semvector, 'rb') as infile:
    semdists = pickle.load(infile)
    
se = SearchCX(A)
A.displaySetup(condenseType='phrase', withNodes=True)

This is Text-Fabric 7.9.0
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

120 features found and 4 ignored
  0.00s loading features ...
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used
  5.21s All features loaded/computed - for details use loadLog()


# Dataset

The current cx dataset excludes time phrases that have gaps inside. These will be analyzed at a later stage due to their complexity. Let's get a sense for how many there are and what is included in the analysis set. The `timephrase` object is a custom object built from the ETCBC phrase object. It makes several corrections as well as fusions of the time phrases. The `timephrase` object is what the Construction classes are built upon.

In [2]:
all_times = A.search('timephrase', shallow=True)

  0.00s 3879 results


In [3]:
analyzed_times = set(phrase2cxs.keys())

In [4]:
unanalyzed_times = collections.Counter()

for time in all_times - analyzed_times:
    surface = tokenize_surface(time, api)
    unanalyzed_times[surface] += 1
    
print(sum(unanalyzed_times.values()), 'times not analyzed...')
print()
print("summary:")

unanalyzed_times = convert2pandas(unanalyzed_times)

unanalyzed_times

17 times not analyzed...

summary:


,Total
ב.ה.שׁנה.ה.ראשׁונה.ב.ה.חדשׁ.ה.ראשׁון,1
מן.ה.יום.ו.עד.ה.יום.ה.זה,1
מספר.ה.ימים.שׁלשׁ.מאות.ו.תשׁעים.יום,1
ב.יום.ו.עד.ה.יום.ה.זה,1
כ.ימות.שׁנות,1
מ.ימי.ה.שׁפטים.ו.כל.ימי.מלכי.ישׂראל.ו.מלכי.יהודה,1
תמיד.מ.רשׁית.ה.שׁנה.ו.עד.אחרית.שׁנה,1
ל.מן.ה.יום.עד.ה.יום.ה.זה,1
ב.שׁלושׁה.עשׂר.יום.בו.ב.ה.יום,1
ל.מן.ה.יום.ו.עד.ה.יום.ה.זה,1


# Basic Exploration

The most basic clustering for time phrases is their surface forms. What are the most common types?

In [5]:
analyzed_time_forms = collections.Counter()

for time in analyzed_times:
    surface = tokenize_surface(time, api)
    analyzed_time_forms[surface] += 1
    
analyzed_time_forms = convert2pandas(analyzed_time_forms)

In [6]:
print(f'{analyzed_time_forms.shape[0]} unique surface forms found')

1149 unique surface forms found


In [7]:
top = 20
print(f'showing top {top} surface forms')
analyzed_time_forms.head(top)

showing top 20 surface forms


,Total
עתה,342
ב.ה.יום.ה.הוא,203
ה.יום,191
ל.עולם,85
ב.ה.בקר,78
עד.ה.יום.ה.זה,71
ב.יום,68
אז,66
שׁבעת.ימים,63
עד.עולם,53


This top list accounts for a substantial proportion of all known time adverbials in the dataset:

In [8]:
print(f'ratio of times accounted for in top {top}:')
analyzed_time_forms.head(top).sum()[0] / len(all_times)

ratio of times accounted for in top 20:


0.4132508378448054

# Formal Taxonomy, Dividing the Times

**A time adverbial is defined as any construction that modifies event time.** The construction may be a word, phrase, or even clause. This project is focused on word and phrase level time adverbials. The time adverbials can be divided into two main forms: single-phrase and multi-phrase.

**Single phrase time adverbials contain a single _profiled_ time word.** The "profiled" word is the head of the phrase, following Croft's model of headship as "the primary information bearing unit" (2001: 257ff). In a time adverbial, the head is typically a specialized term that indicates time, though not always (e.g. as is the case with event nouns). Besides the head, single phrasal adverbials can contain other words that modify the head. There are prepositional and non-prepositional varieties of single phrase adverbials. Note that in semantic headship as defined by Croft, it is the object of the preposition, not the preposition itself, which is considered the head of a phrase.

**Multiphrasal time adverbials contain two or more profiled time elements which are coordinated together.** This coordination can come in the form of literal coordination, e.g. with ו, or various kinds of appositional functions, e.g. when multiple prepositions are "stacked" to coordinate a time within a specific position. Multi-phrasal time adverbials appear with any combination of prepositional and non-prepositional forms.

The basic taxonomy looks like so:

```
single-phrase
|     |
|     prepositional
|     |
|     non-prepositional
|
multi-phrase
      |
      prep/non-prep combinations
```

In [9]:
# TO-DO: Generate taxonomy from tags directly

# # build up taxonomy as a directed graph
# taxonomy = nx.DiGraph((
#     ('time', 'single'),
#     ('time', 'multi'),
#     ('single', 'øprep'),
#     ('single', 'prep'),
#     ('øprep', 'bare'),
#     ('prep', 'bare'),
# ))

## A Deductive and Inductive Classification Process

For classifying the current set of time adverbials, we will utilize a process of elimination. That deductive process is aided by the inductive analysis of time adverbial surface form data. In other words, the categories outlined above and to be outlined further below have been identified by looking at the quantities of the surface form counts to see which categories seem to exert influence. The goal is to be guided by the data, but at the same time derive categories which are useful for collocation research.

### Matching (`CXBuilder`) and Searching (`SearchCX`)

The `CXBuilder` class provides methods for testing any number of conditions on a provided element. It can then modify any matched CX, or compile it into a new `Construction` object. 

The tools provided by `CXSearch` can then scan the time adverbials for matches based on the `CXBuilder`'s rules.

### Surface form counting

Surface forms are counted by first being stripped of accentuation, then tokenized along their lexical boundaries, and finally joined on periods. We utilize prominent counts in the inductive side of the process.

### Keeping Track

We maintain a set of constructions which are and are not accounted for as we build and match the conditions.

# Classification

We put together a custom `CXBuilder` for labeling the CXs. For single-phrase constructions, we simply will add an attribute to each CX object: `classification`. The attribute will be a list of class labels that correspond to a position in the taxonomy tree.

**For single-phrase adverbials, the CXbuilder will simply add a classification tag, while a seperate builder will, instead, combine components of multi-phrase constructions into a single analyzed form.**

### Copy and Track Covered Times

In [10]:
class SetSelection:
    """Get sets of CXs based on interesecting sets"""
    def __init__(self, setdict):
        """Initialize.
        
        Args:
            setdict: a dict of string to set mappings
        """
        self.setdict = setdict
    def __getitem__(self, sets):
        """Retrieve overlapping sets.
        
        Args:
            sets: an iterable of strings which are
                the names of the sets to be searched.
        Returns:
            The overlapping set.
        """
        result = set()
        for st in sets:
            if not result:
                result |= self.setdict[st]
            else:
                result = result & self.setdict[st]
        return result
    
    def get_union(self, sets):
        """Return a union of the sets"""
        result = set(
            cx for stname, st in self.setdict.items()
                if stname in sets
                for cx in st
        )
        return result       

def show_classes(classes, classtags, exclude=tuple(), 
                 counts=True, view=False,
                 shuffle=False, end=100, head=50, 
                 **tfkwargs,
                 ):
    """Iterate through overlapping sets and count/display their results"""
    cxs = classes[classtags] - classes.get_union(exclude)
    cl_counts = collections.Counter()
    surface2cx = collections.defaultdict(set)
    
    # tokenize cx and count/store it for review
    for cx in cxs:
        surface = tokenize_surface(cx.slots, api)
        cl_counts[surface] += 1
        surface2cx[surface].add(cx)
        
    # display counts 
    if counts:
        cl_counts = convert2pandas(cl_counts)
        print(cl_counts.sum().sum(), 'results')
        display(cl_counts.head(head))
        
    # display cxs in class tags
    if view is True:
        cxs = list(cxs)
        if shuffle: 
            random.shuffle(cxs)
        for cx in cxs[:end]:
            se.showcx(cx, **tfkwargs)
        return cxs
            
    # display cxs in an iterable of surface forms
    elif view:
        view_list = [
            cx for surf in view
                for cx in surface2cx[surf]
        ]
        if shuffle:
            random.shuffle(view_list)
        for cx in view_list[:end]:
            se.showcx(cx, **tfkwargs)
        return view_list
    
    else:
        return list(cxs)

In [11]:
# build up taxonomy and keep track of todo-cxs
class Tracker:
    """A class for tracking tagged Constructions"""
    
    def __init__(self, classdict, cxset, 
                 exclude=set(),
                 base={'single', 'non_single'}
                ):
        """Initialize Tracker.
        
        Args:
            classdict: dict of class string to set of
                classified CX objects
            cxset: a set of all CXs that are analyzed
            base: the tag which represents the base
                of the analysis
        """
        self.setselect = SetSelection(classdict)
        self.cxset = self.setselect.get_union(base)
        self.classdict = {
            cl:cxs for cl,cxs in classdict.items()
                if cxs & self.cxset
        }
        self.exclude = exclude | base
        
    def tally_classes(self):
        """Return a Counter on classes"""
        count = collections.Counter()
        for cl, cxset in self.classdict.items():
            count[cl] += len(cxset)
        return convert2pandas(count)
        
    def get_found(self):
        """Get CXs that have already been classified."""
        return set(
            cx for classname, cxs in self.classdict.items()
                for cx in cxs if classname not in self.exclude
        )
        
    def get_remaining(self):
        """Get CXs not yet classified."""
        found = self.get_found()
        return self.cxset - found
        
    def remaining_data(self):
        """Make a count dict of all remaining forms"""
        remaining = self.get_remaining()
        count = collections.Counter()
        form2cxs = collections.defaultdict(set)
        for cx in remaining:
            slots = cx.slots
            surface = tokenize_surface(slots, api) 
            count[surface] += 1
            form2cxs[surface].add(cx)
        return (count, form2cxs)
        
    def remaining_forms(self):
        """Retrieve a sorted count of remaining CX surface forms"""
        count,x = self.remaining_data()
        return convert2pandas(count)
    
    def see_remaining(self, forms, end=10, shuffle=False, **tf_kwargs):
        """Display remaining cxs that are fed in"""
        x,form2cxs = self.remaining_data()
        cxs = list(
            cx for form in forms
                for cx in form2cxs[form]
        )
        if shuffle:
            random.shuffle(cxs)
        for cx in cxs[:end]:
            se.showcx(cx, **tf_kwargs)
        return cxs
    
    def percent(self, n1, total):
        """Calculate ratio"""
        return round(n1/total, 2) * 100
        
    def prog(self, head=10):
        """Report progress dynamically."""
        
        # report progress
        to_do = len(self.get_remaining())
        done = len(self.get_found())
        done_progress = self.percent(done, done+to_do)
        todo_progress = self.percent(to_do, done+to_do)
        print(f'{done_progress}% ({done}) classified')
        print(f'{todo_progress}% ({to_do}) unclassified')
        
        # report class counts
        print()
        print(f'Class counts:')
        class_counts = self.tally_classes()
        display(class_counts)
        
        # report forms of unclassified CXs
        print()
        remain_forms = self.remaining_forms()
        print(f'Top {head} unclassified surface forms') 
        display(remain_forms.head(head))

### CXBuilders

In [37]:
# import sys

# sys.path.append('../cxs/')

# from phrase_classes import SinglePhrase

In [38]:
# # PREPROCESSING DATA 

# # copy cxs for modification by builder
# cx_dataset = set(
#     tuple(copy.deepcopy(cx_data))
#         for ph, cx_data in phrase2cxs.items()
# )

# # compile acceptable head lexemes from single-phrased CXs
# good_heads = set()
# for cx_data in cx_dataset:
#     if len(cx_data) == 1:
#         cx = cx_data[0]
#         head = list(cx.getsuccroles('head'))[-1]
#         good_heads.add(F.lex.v(head))
    
# # tag patterns in CXs
# sp = SinglePhrase(cx_dataset, good_heads, A)
# sp.label_cxs()
# print('done')

### Track Progress

In [13]:
track = Tracker(
    sp.class2cx,
    cx_dataset,
    exclude={
        'single', 'component',
        'prep', 'øprep',
    },
)

track.prog(head=25)

100.0% (4297) classified
0.0% (0) unclassified

Class counts:


,Total
single,4297
prep,2604
øprep,1693
definite,1578
bare,1381
component,1017
def_apposition,753
quantified,742
cardinal,555
demonstrative,525



Top 25 unclassified surface forms


,Total


### See remaining forms

In [26]:
test_ph = phrase2cxs[1448639]
test_ph

[CX prep_ph (182104, 182105, 182106, 182107),
 CX prep_ph (182108, 182109, 182110)]

In [15]:
# test = sp.single(test_ph[-1], test_ph)
# test.cases

In [16]:
# se.showcx(test_ph[2])

In [17]:
#remaining = track.see_remaining(['אחת'], condenseType='sentence', end=5, shuffle=True)

### See Results

In [20]:
# classes = SetSelection(sp.class2cx)

# show_cl = show_classes(
#     classes,
#     ('not_single',),
#     #exclude=('single',),
#     end=25,
#     counts=True,
#     view=['כל.ה.ימים'],
#     shuffle=True,
#     condenseType='sentence'
# )

<hr>

## Multi-Phrasals

### To-Do

I have currently written the SinglePhrase builder with only single-phrased examples in mind. However, this misses the important fact that many multi-phrasal CXs will likewise have single-phrasal component parts. I can re-write the single phrase CXBuilder to receive cxs that are also from multi-phrasal items. Yet this approach would be complicated by the fact that not all phrases within a multi-phrasal time construction will be time-oriented. The result would be that I would have skewed class statistics. For example, if a phrase is "למלך" as part of a calendrical CX, it would end up getting counted as a "prepositional" and "definite" time cx. But it is not itself a time CX, only a part of one. But in other cases, the phrase may indeed also be able to function as its own independent time CX, such as in יום ביום. But even this example raises the issue of whether this CX can truly be decomposed into those smaller parts.

It is worth considering whether it is better to:

1. utilize the same rules in the single-phrasal builder to tag constituent phrases in multi-phrasal time CXs
2. re-write many rules separately, at the risk of duplicating logic already handled in single-phrases.

Option 1 has the strength of enforcing consistency across all categories, whereas option 2 has the ability to cater solutions specific to multi-phrasal constructions. 

**I lean toward option 2.** There are likely many phrases, specific to certain constructions, that do not contain heads that are lexicalized for time. These need to be defined individually. It might mean that certain lower level patterns are duplicated. But that would also mean that they are available for future restrictions and modifications specific to multi-phrasal constructions. 

An option 3 might be to require the SinglePhrase Builder as an argument to the MultiPhrase Builder, and use only the patterns which are relevant. This would allow me to take advantage of both situations.

### Update 2019-12-19

I am actually going with option 1. Why? I have found a way to separate time-oriented phrases from non-time oriented phrases by relying on a list of valid head words, harvested from the single-phrase time CXs. This list is augmented by manual additions where necessary. I have added a separate class title for those phrases which are part of a larger whole, `component`. Thus, I am able to maintain a separation from standalone single-phrase CXs and component single-phrase CXs. This is a nice approach. This way, my model accurately reflects the fact that component time CXs are typically composed of CXs that can and often do stand on their own. That is not to say that the situation is wholly compositional. But this does allow a way forward in which I can gradually add more nuance to the formal description. 

In [34]:
phrase2cxs2 = {
    L.u(cx_tuple[0].slots[0],'timephrase')[0]: cx_tuple
        for cx_tuple in cx_dataset
}

In [96]:
class MultiPhrase(CXbuilder):
    """Build composite time adverbial CXs."""
    
    def __init__(self, phrase2cxs, semdistances, tf):
        CXbuilder.__init__(self)
        
        # set up tf methods
        self.tf = tf
        self.F, self.T, self.L = tf.api.F, tf.api.T, tf.api.L
        
        # map cx to phrase node for context retrieval
        self.phrase2cxs = phrase2cxs
        self.cx2phrase = {
            cx:ph 
                for ph in phrase2cxs
                    for cx in phrase2cxs[ph]
        }
        
        self.cxs = (        
        )
        self.dripbucket = (
            self.cxph
        )
        
        self.kind = 'multiphrase'
        
    def getP(self, cx):
        """Retrieve the context of a cx"""
        phrase_node = self.cx2phrase[cx]
        return Positions(
            cx, 
            self.phrase2cxs[phrase_node],
            default=Construction()
        ).get
        
    def cxph(self, cx):
        """Dripbucket function that returns cx as is."""
        return cx
        
    def appo_time(self, cx):
        """Apposition of Time"""
        P = self.getP(cx)
        
        return self.test(
            {
                'element': cx,
                'name': 'appo_time',
                'kind': self.kind,
                'roles': {'head':cx, 'appo': P(1)},
                'conds': {
                    'single in cx.classification':
                        'single' in cx.__dict__.get('classification',[]),
                    'single in P1.classification':
                        'single' in P(1).__dict__.get('classification',[])
                }
            },
        )
    
mp = MultiPhrase(phrase2cxs2, semdists, A)

In [98]:
# all_cxs = [cx for cx_tuple in cx_dataset for cx in cx_tuple]

# se.search(all_cxs, mp.appo_time, show=25)

In [99]:
# test_ph = phrase2cxs2[1447112]

# test_ph

In [101]:
# test = mp.appo_time(test_ph[0])

# se.showcx(test)